In [1]:
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt 
plt.style.use('fivethirtyeight') 
%matplotlib inline 
import seaborn as sns

import warnings 
warnings.filterwarnings("ignore")

In [2]:
datatype={"key": "object","fare_amount": "float64","pickup_datetime": "object", "pickup_longitude": "float64","pickup_latitude": "float64", "dropoff_longitude": "float64", "dropoff_latitude": "float64","passenger_count": "int64" }

df_raw = pd.read_csv("Downloads/archive (15)/train.csv", low_memory=False, nrows=10**5, dtype=datatype,parse_dates=["pickup_datetime"]) 
df_raw.drop(['key'], axis=1,inplace=True)

df_raw.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1
1,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1
2,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2
3,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1
4,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1


In [3]:
import calendar

df_raw['pickup_date']= df_raw['pickup_datetime'].dt.date
df_raw['pickup_day']=df_raw['pickup_datetime'].apply(lambda x:x.day)
df_raw['pickup_hour']=df_raw['pickup_datetime'].apply(lambda x:x.hour)
df_raw['pickup_day_of_week']=df_raw['pickup_datetime'].apply(lambda x:calendar.day_name[x.weekday()]) 
df_raw['pickup_month']=df_raw['pickup_datetime'].apply(lambda x:x.month) 
df_raw['pickup_year']=df_raw['pickup_datetime'].apply(lambda x:x.year)

df_raw.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_date,pickup_day,pickup_hour,pickup_day_of_week,pickup_month,pickup_year
0,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1,2009-06-15,15,17,Monday,6,2009
1,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1,2010-01-05,5,16,Tuesday,1,2010
2,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2,2011-08-18,18,0,Thursday,8,2011
3,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1,2012-04-21,21,4,Saturday,4,2012
4,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1,2010-03-09,9,7,Tuesday,3,2010


In [4]:
#Revoving Outlier Value 
df_raw = df_raw[((df_raw['pickup_longitude']>-78) &

(df_raw['pickup_longitude'] <-70)) & 
((df_raw['dropoff_longitude']>-78) & 
(df_raw['dropoff_longitude'] <-70)) & 
((df_raw['pickup_latitude']> 37) & 
(df_raw['pickup_latitude'] <45)) & 
((df_raw['dropoff_latitude'] > 37) & 
(df_raw['dropoff_latitude'] <45)) & 
(df_raw['passenger_count'] > 0) & 
(df_raw['fare_amount']>=2.5)]

In [5]:
def distance(lat1, loni, lat2, lon2):

    p = 0.017453292519943295 # Pi/180
    a = 0.5-np.cos((lat2 - lat1)* p)/2 + np.cos(lat1*p) *np.cos(lat2*p)* (1- np.cos((lon2-loni)*p))/2 
    return 0.6213712*12742 *np.arcsin(np.sqrt(a))

#add new column to dataframe with distance in miles

df_raw['distance_miles'] = distance(df_raw.pickup_latitude, df_raw.pickup_longitude, \
                                    df_raw.dropoff_latitude, df_raw.dropoff_longitude)

df_raw.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_date,pickup_day,pickup_hour,pickup_day_of_week,pickup_month,pickup_year,distance_miles
0,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1,2009-06-15,15,17,Monday,6,2009,0.640487
1,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1,2010-01-05,5,16,Tuesday,1,2010,5.250670
2,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2,2011-08-18,18,0,Thursday,8,2011,0.863411
3,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1,2012-04-21,21,4,Saturday,4,2012,1.739386
4,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1,2010-03-09,9,7,Tuesday,3,2010,1.242218


In [6]:
df_final = df_raw.copy()
df_final.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_date,pickup_day,pickup_hour,pickup_day_of_week,pickup_month,pickup_year,distance_miles
0,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1,2009-06-15,15,17,Monday,6,2009,0.640487
1,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1,2010-01-05,5,16,Tuesday,1,2010,5.250670
2,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2,2011-08-18,18,0,Thursday,8,2011,0.863411
3,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1,2012-04-21,21,4,Saturday,4,2012,1.739386
4,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1,2010-03-09,9,7,Tuesday,3,2010,1.242218


In [7]:
Variables={'Monday': 1, 'Tuesday': 2, 'Thursday': 3, 'Saturday': 4, 'Wednesday': 5, 'Sunday': 6, 'Friday': 7 }

year={2012: 1,2013: 12, 2011: 3, 2009: 4, 2010:5, 2014: 6, 2015:7}

df_final['pickup_year']=df_final['pickup_year'].map(year)

df_final['pickup_day_of_week']=df_final['pickup_day_of_week'].map(Variables) 
df_final.drop(['pickup_datetime','pickup_date'],axis=1,inplace=True)

df_final.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_day,pickup_hour,pickup_day_of_week,pickup_month,pickup_year,distance_miles
0,4.5,-73.844311,40.721319,-73.841610,40.712278,1,15,17,1,6,4,0.640487
1,16.9,-74.016048,40.711303,-73.979268,40.782004,1,5,16,2,1,5,5.250670
2,5.7,-73.982738,40.761270,-73.991242,40.750562,2,18,0,3,8,3,0.863411
3,7.7,-73.987130,40.733143,-73.991567,40.758092,1,21,4,4,4,1,1.739386
4,5.3,-73.968095,40.768008,-73.956655,40.783762,1,9,7,2,3,5,1.242218


In [8]:
from sklearn.model_selection import train_test_split
X=df_final.drop(['fare_amount'],axis=1)
y=df_final[['fare_amount']]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [9]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_depth=5, random_state=0, n_estimators=200) 
rf.fit(X_train, y_train) 
y_pred = rf.predict(X_test) 

In [12]:
print("Enter Your Information to Predict Fare: ")

Pickup_Coordinates=str(input("Enter Pickup Coordinates: ")) 
Drop_Off_Coordinates=str(input("Enter Drop-off Coordinates: "))

pickup_latitude = float(Pickup_Coordinates.split(',')[0])

pickup_longitude = float(Pickup_Coordinates.split(',')[1])

dropoff_latitude = float(Drop_Off_Coordinates.split(',')[0]) 
dropoff_longitude = float(Drop_Off_Coordinates.split(',')[1])

Enter_Date=input("Enter Date (DD/MM/YEAR)")
passenger_count = int(input('Enter Number of Passengers: '))
pickup_day=int(Enter_Date.split('/')[0])
pickup_hour = int(input('Enter Hour: '))
pickup_day_of_week = int(input('Enter Week: '))

pickup_month = int(Enter_Date.split('/')[1])

pickup_year = int(Enter_Date.split('/')[2])

distance_miles=distance(pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude)

my_array = np.array([pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude, passenger_count, pickup_day, pickup_hour, pickup_day_of_week, pickup_month, pickup_year, distance_miles])

print('Fare Amount ($): ',rf.predict(my_array.reshape(1,-1)))

Enter Your Information to Predict Fare: 
Enter Pickup Coordinates: -73.844311,40.721319	
Enter Drop-off Coordinates: -73.841610,40.712278
Enter Date (DD/MM/YEAR)29/02/2024
Enter Number of Passengers: 2
Enter Hour: 4
Enter Week: 2
Fare Amount ($):  [29.01532506]


In [13]:
import pickle

# Save the model to a file
with open('random_forest_model.pkl', 'wb') as file:
    pickle.dump(rf, file)

    

In [15]:
# Load the model from the file
with open('random_forest_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)


In [16]:
from IPython.display import FileLink

FileLink('random_forest_model.pkl')


C:\Users\hp\random_forest_model.pkl